<a href="https://colab.research.google.com/github/naumanHadi/Deep_NN_GeneratorFailures/blob/main/Excel_LLM_LangChainRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain langchain-community chromadb sentence-transformers openpyxl tiktoken gradio

import os, glob, pandas as pd, numpy as np
from langchain_community.document_loaders import DataFrameLoader
from langchain.docstore.document import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import BaseRetriever
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import openai
from google.colab import userdata
import os, glob,re
import pandas as pd
from diffusers import DiffusionPipeline
from transformers import pipeline
from datasets import load_dataset
import soundfile as sf
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma

# Retrieval + Memory + Chains
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 9.5 MB/s eta 0:00

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_excel("/content/drive/MyDrive/Python_Course/Allagi/Combined_Violations.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12898 entries, 0 to 12897
Data columns (total 91 columns):
 #   Column                                                                                     Non-Null Count  Dtype         
---  ------                                                                                     --------------  -----         
 0   ID                                                                                         12898 non-null  int64         
 1   Usid                                                                                       12898 non-null  int64         
 2   Quad                                                                                       12898 non-null  int64         
 3   Region                                                                                     12898 non-null  object        
 4   Market                                                                                     12898 non-null  object        
 5

Loading all the data from Google Drive (Combined Violations)

In [ ]:
#df["SSV Held Up Reasons - Ericsson RF ( More Details)"].head(500)
df["SSV Held Up Reasons - Ericsson RF"].head(5)


,SSV Held Up Reasons - Ericsson RF
0,
1,
2,
3,
4,


In [ ]:
# =========================
# STEP 2 — Load, Clean, Prepare (NO AUTO-DROP)
# =========================

# LangChain Document import (compatible with older/newer LC)
try:
    from langchain_core.documents import Document
except Exception:
    from langchain.docstore.document import Document

import os, re, glob, difflib, unicodedata
import pandas as pd

# --- Set this ONCE. Can be a single Excel file OR a folder path ---
DATA_PATH = "/content/drive/MyDrive/Python_Course/Allagi/Combined_Violations.xlsx"  # or folder

# --- Build mode ---
MODE    = "ROW_MODE"   # "ROW_MODE" now; switch to "PROFILE_MODE" if you later have multiple rows per Usid
KEY_COL = "Usid"       # used only in PROFILE_MODE

# --- Your intended text-heavy columns (as you named them) ---
WANT_TEXT_COLS_RAW = [
    "Turf Vendor Additional Description/Comments",
    "Pending HO Reasons (TV/Market PM)",
    "SSV Held Up Reasons - Ericsson RF",
    "SSV Held Up Reasons - Ericsson RF ( More Details)",
    # Optionally include if sentence-like:
    # "True Handover Status",
]

# --- Metadata to keep (filtering; not embedded) ---
META_COLS = [
    "ID", "Usid", "Quad", "Region", "Market", "SubMarket",
    "CivilSupplier", "SwapType", "Completed", "ClosedOut",
    "CutoverDate", "SSV Hold ( Removal) Date",
    "True Handover Status", "__file", "__sheet",
]

# --- Columns you NEVER want used in RAG (excluded from embeddings & metadata) ---
RAG_EXCLUDE = {
    "Flag2 - Ready To Close (RTC)",
    "Pending Handover Status (Flag1)",
    "Aging Optimization Reports",
    "Flag3 ( DowtimeAlert)",
    "Flag3 (DowntimeAlert)",
    "5G Traffic",
    "5G  Site True Flag",
    "Staggered Launch (Priority))",
    "Staggered Launch (Priority)",
    "5G Traffic Flag",
    "Delayed HO Flag",
    "Auto Ticketing ( Extended Partial Outage > 100 , Performance Issues, 5G Staggered Launch)",
}

# --- Optional: explicit white/black lists (kept here but not needed if you don't want any drop) ---
KEEP_COLS = []   # e.g., ["ID","Usid","Region","Market"]
DROP_COLS = []   # e.g., ["Unnamed: 0"]

# --- Header normalization / renames ---
RENAME_MAP = {
    "Staggered Launch (Priority))": "Staggered Launch (Priority)",
    "Flag3 ( DowtimeAlert)": "Flag3 (DowntimeAlert)",
    "True Handover Status ": "True Handover Status",  # trim trailing space
    "SSV Hold Flag ": "SSV Hold Flag"
}

def _clean_headers(cols):
    seen = {}
    new_cols = []
    for c in cols:
        nc = re.sub(r"\s+", " ", str(c or "")).strip()
        if nc.startswith("Unnamed"):
            nc = "Unnamed"
        nc = RENAME_MAP.get(nc, nc)
        if nc in seen:
            seen[nc] += 1
            nc = f"{nc}__{seen[nc]}"
        else:
            seen[nc] = 1
        new_cols.append(nc)
    return new_cols

def _read_excels(path):
    frames = []
    if os.path.isfile(path) and path.lower().endswith((".xlsx", ".xls")):
        file_list = [path]
    else:
        file_list = glob.glob(f"{path}/**/*.xlsx", recursive=True) + \
                    glob.glob(f"{path}/**/*.xls", recursive=True)
    for f in file_list:
        try:
            xls = pd.ExcelFile(f)
            for sheet in xls.sheet_names:
                df = pd.read_excel(f, sheet_name=sheet, dtype=str)
                df["__file"] = os.path.basename(f)
                df["__sheet"] = sheet
                frames.append(df)
        except Exception as e:
            print("Skipping:", f, "->", e)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

# --- 2.1 Load ---
raw = _read_excels(DATA_PATH)
print("Raw shape:", raw.shape)

# --- 2.2 Clean headers ---
df = raw.copy()
df.columns = _clean_headers(df.columns)

# --- 2.3 Drop explicit bad columns ONLY (no auto-drop anywhere else) ---
if DROP_COLS:
    df = df.drop(columns=[c for c in DROP_COLS if c in df.columns], errors="ignore")

# --- 2.4 KEEP_COLS whitelist (optional) ---
if KEEP_COLS:
    keep = [c for c in KEEP_COLS if c in df.columns]
    for sys_tag in ["__file", "__sheet"]:
        if sys_tag in df.columns and sys_tag not in keep:
            keep.append(sys_tag)
    if keep:
        df = df[keep]

# --- 2.5 Fill & normalize whitespace (no deprecated applymap) ---
df = df.fillna("")
df = df.apply(lambda s: (
    s.astype(str)
     .str.replace(r"\s+", " ", regex=True)
     .str.strip()
))

# --- 2.6 Resolve desired TEXT_COLS against cleaned headers (fuzzy-safe) ---
def _canon(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[\(\)\-_/]+", " ", s)   # drop common separators
    s = re.sub(r"\s+", " ", s).strip()
    return s

canon_to_real = {_canon(c): c for c in df.columns}
canon_cols = list(canon_to_real.keys())

resolved_text_cols = []
mapping_report = []
for want in WANT_TEXT_COLS_RAW:
    want_key = _canon(want)
    if want_key in canon_to_real:
        real = canon_to_real[want_key]
        resolved_text_cols.append(real)
        mapping_report.append(f"✔ '{want}' → '{real}'")
    else:
        close = difflib.get_close_matches(want_key, canon_cols, n=1, cutoff=0.65)
        if close:
            real = canon_to_real[close[0]]
            resolved_text_cols.append(real)
            mapping_report.append(f"~ '{want}' → '{real}' (fuzzy)")
        else:
            mapping_report.append(f"✘ '{want}' → NOT FOUND")

print("\n".join(mapping_report))

# keep only valid, non-excluded text cols
TEXT_COLS = [c for c in resolved_text_cols if c in df.columns and c not in RAG_EXCLUDE and c not in {"__file","__sheet"}]

# If all chosen text cols are empty strings for all rows, we still keep them (per your "don't drop")
# but we warn so you know why some rows may produce no text.
def _nonempty_ratio(col):
    return (df[col].astype(str).str.strip() != "").mean() if col in df.columns else 0.0

for c in TEXT_COLS:
    ratio = _nonempty_ratio(c)
    if ratio == 0.0:
        print(f"⚠ Column '{c}' appears entirely empty after cleaning; docs from those rows may be blank.")

# Validate metadata list against df + exclusions
META_COLS = [c for c in META_COLS if c in df.columns and c not in RAG_EXCLUDE]

def _row_to_narrative(row_dict, text_cols):
    """Turn selected text columns into a compact narrative with section headers; skip blanks."""
    parts = []
    for c in text_cols:
        val = str(row_dict.get(c, "")).strip()
        if val and val not in {"-", "N/A", "NA"}:
            parts.append(f"## {c}\n{val}")
    return "\n\n".join(parts)

# --- 2.7 Build documents ---
docs = []

if MODE == "ROW_MODE":
    for r in df.itertuples(index=False):
        row = r._asdict()
        text = _row_to_narrative(row, TEXT_COLS)
        if not text:
            # keep behavior: if a row has no text in TEXT_COLS, skip building a doc for that row
            # (this avoids blank docs in your vector store)
            continue
        meta = {k: row.get(k, "") for k in META_COLS}
        meta["_doc_id"] = str(row.get("Usid", "")) or str(row.get("ID", ""))
        docs.append(Document(page_content=text, metadata=meta))

elif MODE == "PROFILE_MODE":
    if KEY_COL not in df.columns:
        raise ValueError(f"PROFILE_MODE requires KEY_COL '{KEY_COL}' in the data.")
    grouped = df.groupby(KEY_COL, dropna=False)
    for key_val, g in grouped:
        sections = []
        for c in TEXT_COLS:
            vals = [v for v in g[c].astype(str).tolist() if v.strip() and v not in {"-", "N/A", "NA"}]
            if not vals:
                continue
            seen = set()
            uniq = [v for v in vals if not (v in seen or seen.add(v))]
            sections.append(f"## {c}\n" + "\n\n".join(uniq))
        if not sections:
            continue
        header = [f"# Profile — {KEY_COL}: {key_val}"]
        for k in ["Market", "Region", "CivilSupplier", "True Handover Status"]:
            if k in g.columns:
                vals = [v for v in g[k].astype(str).unique().tolist() if v.strip()]
                if vals:
                    header.append(f"- {k}: {', '.join(vals[:3])}")
        text = "\n".join(header) + "\n\n" + "\n\n".join(sections)
        last = g.tail(1).to_dict(orient="records")[0]
        meta = {k: last.get(k, "") for k in META_COLS}
        meta[KEY_COL] = key_val
        meta["_doc_id"] = str(key_val)
        docs.append(Document(page_content=text, metadata=meta))
else:
    raise ValueError("MODE must be 'ROW_MODE' or 'PROFILE_MODE'")

print(f"✅ Cleaned shape: {df.shape}")
print(f"✅ Built {len(docs)} documents")
print(f"🧾 TEXT_COLS used for embeddings: {TEXT_COLS}")
print(f"🏷️ Metadata columns (for filters): {META_COLS}")
if docs:
    print("🔎 Example doc metadata:", docs[0].metadata)
    print("🔎 Example doc preview:\n", docs[0].page_content[:600], "...")
else:
    print("No documents built — verify TEXT_COLS and input data.")


Raw shape: (12898, 93)
✔ 'Turf Vendor Additional Description/Comments' → 'Turf Vendor Additional Description/Comments'
✔ 'Pending HO Reasons (TV/Market PM)' → 'Pending HO Reasons (TV/Market PM)'
✔ 'SSV Held Up Reasons - Ericsson RF' → 'SSV Held Up Reasons - Ericsson RF'
✔ 'SSV Held Up Reasons - Ericsson RF ( More Details)' → 'SSV Held Up Reasons - Ericsson RF ( More Details)'
✅ Cleaned shape: (12898, 93)
✅ Built 0 documents
🧾 TEXT_COLS used for embeddings: ['Turf Vendor Additional Description/Comments', 'Pending HO Reasons (TV/Market PM)', 'SSV Held Up Reasons - Ericsson RF', 'SSV Held Up Reasons - Ericsson RF ( More Details)']
🏷️ Metadata columns (for filters): ['ID', 'Usid', 'Quad', 'Region', 'Market', 'SubMarket', 'CivilSupplier', 'SwapType', 'Completed', 'ClosedOut', 'CutoverDate', 'SSV Hold ( Removal) Date', 'True Handover Status', '__file', '__sheet']
No documents built — verify TEXT_COLS and input data.


In [ ]:
# =========================
# STEP 2.10 — Build row docs with ID header + safe splitting
# =========================
# Requires: df, TEXT_COLS, META_COLS from earlier; Document class imported.
# Produces: row_docs, split_docs

import re
from langchain.text_splitter import TokenTextSplitter
import tiktoken

# --- Config ---
INCLUDE_EMPTY_ROWS = True   # <-- keep even rows with no text in TEXT_COLS
MAX_ROW_TOKENS     = 2000
SPLIT_SIZE         = 700
SPLIT_OVERLAP      = 60

# What to surface in the header (embed these so numeric lookups work)
HEADER_KEYS = [
    "Usid", "ID", "Region", "Market", "SubMarket",
    "True Handover Status", "CutoverDate", "CivilSupplier", "SwapType",
    "Completed", "ClosedOut"
]

_MISS = {"", "0", "-", "na", "n/a", "none", "null"}
def _norm(x: str) -> str:
    s = re.sub(r"\s+", " ", str(x or "")).strip()
    return "" if s.lower() in _MISS else s

def _header_block(row_dict: dict) -> str:
    parts = []
    for k in HEADER_KEYS:
        if k in row_dict:
            v = _norm(row_dict[k])
            if v: parts.append(f"{k}: {v}")
    # Also ensure canonical numeric tokens exist exactly once
    usid = _norm(row_dict.get("Usid", ""))
    if usid and not any(p.startswith("Usid:") or p.startswith("USID:") for p in parts):
        parts.insert(0, f"USID: {usid}")
    _id = _norm(row_dict.get("ID", ""))
    if _id and not any(p.startswith("ID:") for p in parts):
        parts.insert(1 if usid else 0, f"ID: {_id}")
    return "## Keys\n" + " | ".join(parts) if parts else "## Keys\n"

def _section(col: str, val: str) -> str:
    v = _norm(val)
    return f"## {col}\n{v}\n" if v else ""

# Build one page_content per row
def _row_to_page(row):
    r = row.to_dict()
    header = _header_block(r)
    # sections for your 4 text-heavy columns
    body_parts = []
    for col in TEXT_COLS:
        if col in r:
            body_parts.append(_section(col, r[col]))
    body = "\n".join([p for p in body_parts if p])
    if not body.strip():
        if not INCLUDE_EMPTY_ROWS:
            return ""  # signal to drop this row
        body = "## Notes\n(No detailed comments available.)"
    return header + "\n\n" + body

# Create one Document per row (with metadata), then split long ones
row_docs = []
for _, row in df.iterrows():
    page = _row_to_page(row)
    if not page:   # dropped due to INCLUDE_EMPTY_ROWS=False and no text
        continue
    md = {k: row[k] for k in META_COLS if k in df.columns}
    # ensure retrieval keys exist in metadata too
    md["_doc_id"] = md.get("Usid") or md.get("ID") or ""
    md["_chunk_id"] = "whole"
    row_docs.append(Document(page_content=page, metadata=md))

print(f"Built {len(row_docs)} row docs (header + sections)")

# --- Conditional split to avoid embedding token errors ---
enc = tiktoken.get_encoding("cl100k_base")
def ntoks(text: str) -> int: return len(enc.encode(text or ""))

token_splitter = TokenTextSplitter(chunk_size=SPLIT_SIZE, chunk_overlap=SPLIT_OVERLAP)

split_docs = []
kept = split_ct = 0
for d in row_docs:
    if ntoks(d.page_content) <= MAX_ROW_TOKENS:
        split_docs.append(d); kept += 1
    else:
        # optional pre-split by '## ' boundaries so fields stay intact
        paragraphs = re.split(r"\n(?=## )", d.page_content)
        pre_docs = [type(d)(page_content=p, metadata={**d.metadata, "_chunk_id": f"p{i+1}"}) for i, p in enumerate(paragraphs) if p.strip()]
        chunks = token_splitter.split_documents(pre_docs)
        # ensure chunk ids are unique and deterministic
        for i, c in enumerate(chunks):
            c.metadata["_chunk_id"] = c.metadata.get("_chunk_id", "whole") + f":c{i+1}"
        split_docs.extend(chunks); split_ct += 1

print(f"Rows kept whole: {kept}, rows split: {split_ct}, total chunks: {len(split_docs)}")

# Sanity peek
if split_docs:
    s = split_docs[0]
    print("Sample metadata:", s.metadata)
    print("Sample content:\n", (s.page_content[:400] + "...") if len(s.page_content) > 400 else s.page_content)
else:
    print("No documents created; check TEXT_COLS and HEADER_KEYS.")


Built 12898 row docs (header + sections)
Rows kept whole: 12892, rows split: 6, total chunks: 12956
Sample metadata: {'ID': '12997', 'Usid': '208177', 'Quad': '3', 'Region': 'WEST', 'Market': 'PACIFIC NORTHWEST', 'SubMarket': 'OREGON', 'CivilSupplier': '', 'SwapType': 'Cold', 'Completed': 'False', 'ClosedOut': '', 'CutoverDate': '2025-08-22 00:00:00', 'SSV Hold ( Removal) Date': '', 'True Handover Status': '', '__file': 'Combined_Violations.xlsx', '__sheet': 'Sheet1', '_doc_id': '208177', '_chunk_id': 'whole'}
Sample content:
 ## Keys
Usid: 208177 | ID: 12997 | Region: WEST | Market: PACIFIC NORTHWEST | SubMarket: OREGON | CutoverDate: 2025-08-22 00:00:00 | SwapType: Cold | Completed: False

## Notes
(No detailed comments available.)


In [ ]:
!pip -q install -U langchain-community chromadb tiktoken tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.1 MB/s eta 0:00:00


In [ ]:
# =========================
# STEP 3 — Embed with OpenAI + add to Chroma (fresh DB, progress bars)
# =========================
# %pip -q install -U langchain-openai langchain-community chromadb tiktoken tqdm

# 3.0 Imports
try:
    from langchain_openai import OpenAIEmbeddings          # preferred
except Exception:
    from langchain.embeddings.openai import OpenAIEmbeddings  # legacy

from langchain_community.vectorstores import Chroma
from tqdm.auto import tqdm
import os, shutil, datetime as dt
import tiktoken

# 3.1 Use today's chunks
source_docs = split_docs if "split_docs" in globals() and split_docs else docs
assert source_docs and len(source_docs) > 0, "split_docs/docs is empty — run Step 2.10 first."

# 3.2 API key
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY") or os.environ.get("OPENAI_API_KEY", "")
assert os.environ.get("OPENAI_API_KEY"), "Set OPENAI_API_KEY."

# 3.3 Fresh persist dir + collection
excel_folder  = os.path.dirname(DATA_PATH)

# ⚠️ Force a brand-new DB folder to avoid 'no such table: tenants'
PERSIST_DIR   = os.path.join(excel_folder, "chroma_db_hdr")  # new folder name
if os.path.exists(PERSIST_DIR):
    print(f"🧹 Removing old DB at {PERSIST_DIR} …")
    shutil.rmtree(PERSIST_DIR)
os.makedirs(PERSIST_DIR, exist_ok=True)

collection_name = f"tv_profiles_hdr_{dt.datetime.now():%Y%m%d_%H%M}"
print(f"📂 Chroma DB: {PERSIST_DIR}\n🗂️ Collection: {collection_name}")

# 3.4 Embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # or 3-large

# 3.5 Create collection (with fallback to /content/ if Drive locks)
def open_chroma_or_fallback():
    try:
        return Chroma(persist_directory=PERSIST_DIR,
                      collection_name=collection_name,
                      embedding_function=embeddings)
    except Exception as e:
        print("⚠️ Persist init failed, falling back to local runtime:", e)
        fb = f"/content/chroma_hdr_{dt.datetime.now():%Y%m%d_%H%M%S}"
        os.makedirs(fb, exist_ok=True)
        print("📦 Using fallback DB at:", fb)
        return Chroma(persist_directory=fb,
                      collection_name=collection_name,
                      embedding_function=embeddings)

chroma_vs = open_chroma_or_fallback()

# 3.6 Token-safe batching + progress
enc = tiktoken.get_encoding("cl100k_base")
MAX_TOKENS_PER_REQUEST = 240_000
MAX_DOCS_PER_BATCH     = 64

def batch_docs_by_tokens(docs, max_tokens=MAX_TOKENS_PER_REQUEST, max_items=MAX_DOCS_PER_BATCH):
    batch, toks = [], 0
    for d in docs:
        t = len(enc.encode(d.page_content or ""))
        if t >= max_tokens:
            if batch: yield batch; batch, toks = [], 0
            yield [d]; continue
        if batch and (toks + t > max_tokens or len(batch) >= max_items):
            yield batch; batch, toks = [], 0
        batch.append(d); toks += t
    if batch: yield batch

total_docs   = len(source_docs)
total_tokens = sum(len(enc.encode(d.page_content or "")) for d in source_docs)
added = 0

with tqdm(total=total_docs,  desc="Embedding & adding (docs)",   unit="doc", dynamic_ncols=True) as p_docs, \
     tqdm(total=total_tokens, desc="Embedding & adding (tokens)", unit="tok", dynamic_ncols=True) as p_toks:
    for batch in batch_docs_by_tokens(source_docs):
        bdocs = list(batch)
        chroma_vs.add_documents(bdocs)
        added += len(bdocs)
        p_docs.update(len(bdocs))
        p_toks.update(sum(len(enc.encode(d.page_content or "")) for d in bdocs))

print(f"✅ Added {added} vectors. Total now: {chroma_vs._collection.count()}")

# 3.7 Retriever
retriever = chroma_vs.as_retriever(search_type="mmr", search_kwargs={"k": 8, "fetch_k": 32})
print("✅ Retriever ready.")

# 3.8 Smoke test
q = "Tell me about 59919"
hits = retriever.get_relevant_documents(q)
print("\nTop hit metadata:", hits[0].metadata if hits else "—")
print("Top hit preview:", hits[0].page_content[:200] + "..." if hits else "—")


🧹 Removing old DB at /content/drive/MyDrive/Python_Course/Allagi/chroma_db_hdr …
📂 Chroma DB: /content/drive/MyDrive/Python_Course/Allagi/chroma_db_hdr
🗂️ Collection: tv_profiles_hdr_20250813_1539


/tmp/ipython-input-839399258.py:40: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # or 3-large
/tmp/ipython-input-839399258.py:45: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(persist_directory=PERSIST_DIR,


Embedding & adding (docs):   0%|          | 0/12956 [00:00<?, ?doc/s]

Embedding & adding (tokens):   0%|          | 0/1399669 [00:00<?, ?tok/s]

✅ Added 12956 vectors. Total now: 12956
✅ Retriever ready.


/tmp/ipython-input-839399258.py:97: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  hits = retriever.get_relevant_documents(q)



Top hit metadata: {'True Handover Status': '', 'ClosedOut': '', 'Completed': 'False', 'SubMarket': 'PHILADELPHIA', 'ID': '8093', '_doc_id': '299691', '__sheet': 'Sheet1', 'SSV Hold ( Removal) Date': '', 'SwapType': 'Cold', '__file': 'Combined_Violations.xlsx', 'Quad': '3', 'CutoverDate': '2025-08-15 00:00:00', 'Market': 'E. PA/S. NJ/DE', 'CivilSupplier': 'Jacobs', 'Usid': '299691', '_chunk_id': 'whole', 'Region': 'EAST'}
Top hit preview: ## Keys
Usid: 299691 | ID: 8093 | Region: EAST | Market: E. PA/S. NJ/DE | SubMarket: PHILADELPHIA | CutoverDate: 2025-08-15 00:00:00 | CivilSupplier: Jacobs | SwapType: Cold | Completed: False

## Not...


In [ ]:
# =========================
# STEP 4 — LLM + RAG chains (flexible: one-liner for fields, summary otherwise)
# =========================

# 4.1 Import ChatOpenAI (new path first; fallback for older stacks)
try:
    from langchain_openai import ChatOpenAI
except Exception:
    from langchain.chat_models import ChatOpenAI  # older LangChain

from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# --- ID/USID filter helper (metadata-aware retrieval) ---
import re
def _build_filter_from_question(q: str):
    """Extract a USID or ID from the question and build a Chroma filter."""
    q = str(q)
    m = re.search(r'\b(?:USID\s*[:#]?\s*)?(\d{3,})\b', q, re.I)  # matches 'USID 59919' or just '59919'
    if m: return {"Usid": m.group(1)}
    m = re.search(r'\bID\s*[:#]?\s*(\d{3,})\b', q, re.I)         # matches 'ID 12011'
    if m: return {"ID": m.group(1)}
    return None

# 4.2 Pick your LLM
LLM_MODEL = "gpt-4o"  # or "gpt-4o-mini" for cheaper/faster
llm = ChatOpenAI(model=LLM_MODEL, temperature=0)

# 4.3 Flexible prompt:
#     - If the user asks for ONE attribute (Market/Region/SubMarket/CivilSupplier/True Handover Status/CutoverDate/SwapType/Completed/ClosedOut),
#       respond EXACTLY "<Attribute>: <value>" (no extra words).
#     - Otherwise, write a concise narrative (3–6 sentences) that mentions CivilSupplier & True Handover Status,
#       and summarizes ONLY:
#         • SSV Held Up Reasons – Ericsson RF ( More Details)  (Ericsson comments)
#         • Turf Vendor Additional Description/Comments         (Civil supplier comments)
#       If both comment fields are missing, say: "No detailed reasons/comments provided."
def build_flexible_prompt() -> ChatPromptTemplate:
    system = (
   "You are a helpful analyst for the AT&T Allagi telecom deployment project.\n"
        # Small talk bypass
        "If the user's message is a greeting, acknowledgement, thanks, or farewell "
        "(e.g., 'hi', 'hello', 'thanks', 'ok', 'bye'), reply briefly and naturally "
        "WITHOUT using any context.\n\n"

        # Grounded RAG behavior
        "Otherwise, answer STRICTLY from the provided context (Excel row profiles). "
        "If not in the context, reply exactly: \"I don't know based on the provided context.\" "
        "Treat 0, '-', 'NA', and empty as missing. Preserve numeric identifiers (USID, ID) verbatim.\n\n"

        # One-liner mode for single attribute lookups
        "If the user's question asks for ONE attribute among "
        "[Market, Region, SubMarket, CivilSupplier, True Handover Status, CutoverDate, "
        "SwapType, Completed, ClosedOut], respond with EXACTLY '<Attribute>: <value>' "
        "and nothing else.\n\n"

        # Summary mode for everything else
        "Otherwise, write a concise narrative summary (3–6 sentences) that:\n"
        "• Summarizes ONLY these two comment fields when present:\n"
        "  - SSV Held Up Reasons – Ericsson RF ( More Details)\n"
        "  - Turf Vendor Additional Description/Comments\n"
        "If both comment fields are missing, add one sentence: 'No detailed reasons/comments provided.' "
        "Do NOT use bullet points or tables."
    )
    return ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", "Question: {question}\n\nContext:\n{context}")
    ])

qa_prompt = build_flexible_prompt()

# Keep numeric IDs intact during follow-ups
CONDENSE_PROMPT = PromptTemplate.from_template(
    "Rewrite the user's follow-up into a single, standalone search query. "
    "PRESERVE any numeric identifiers exactly (e.g., USID, ID). "
    "Don't answer the question; only rewrite it. Keep it brief and keyword-rich. "
    "If the follow-up refers to 'this/that site', infer the USID/ID from the chat history if present.\n\n"
    "Chat history:\n{chat_history}\n\n"
    "Follow-up question: {question}\n\n"
    "Standalone query:"
)

# 4.4 A) Stateless RAG (strictly grounded to retrieved chunks)
stateless_qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,               # from Step 3
    chain_type="stuff",
    chain_type_kwargs={"prompt": qa_prompt},
    return_source_documents=True
)

def ask_once(question: str, k: int = 8):
    flt = _build_filter_from_question(question)
    prev_kwargs = dict(stateless_qa.retriever.search_kwargs)  # save
    try:
        new_kwargs = {"k": k}
        if flt: new_kwargs["filter"] = flt
        stateless_qa.retriever.search_kwargs.update(new_kwargs)
        out = stateless_qa.invoke({"query": question})
        answer  = out["result"]
        sources = out.get("source_documents", [])
    finally:
        stateless_qa.retriever.search_kwargs = prev_kwargs  # restore

    print("🧠 Answer:\n", answer, "\n")
    print("📚 Sources:")
    for i, d in enumerate(sources[:5], 1):
        preview = d.page_content[:200] + ("..." if len(d.page_content) > 200 else "")
        print(f"[{i}] {d.metadata} -> {preview}")
    return answer, sources

# 4.5 B) Conversational RAG (memory fixed with output_key/input_key)
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="question",     # what the chain expects as input
    output_key="answer",      # which output to store in memory
    return_messages=True
)

# Use the SAME flexible prompt + the condense prompt inside the conversational chain
conv_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    condense_question_prompt=CONDENSE_PROMPT,
    combine_docs_chain_kwargs={"prompt": qa_prompt},
)

def chat_once(question: str, k: int = 8):
    flt = _build_filter_from_question(question)
    prev_kwargs = dict(conv_chain.retriever.search_kwargs)  # save
    try:
        new_kwargs = {"k": k}
        if flt: new_kwargs["filter"] = flt
        conv_chain.retriever.search_kwargs.update(new_kwargs)
        out = conv_chain.invoke({"question": question})
        answer  = out["answer"]
        sources = out.get("source_documents", [])
    finally:
        conv_chain.retriever.search_kwargs = prev_kwargs  # restore

    print("💬 Chat Answer:\n", answer, "\n")
    print("📚 Sources:")
    for i, d in enumerate(sources[:5], 1):
        preview = d.page_content[:200] + ("..." if len(d.page_content) > 200 else "")
        print(f"[{i}] {d.metadata} -> {preview}")
    return answer, sources

# 4.6 Quick smoke tests
print("— Stateless QA —")
_ = ask_once("What market is USID 59919 in?", k=8)          # should return one-liner
_ = ask_once("Summarize USID 59919.", k=12)                 # should return narrative

print("\n— Conversational QA —")
_ = chat_once("Tell me about 59919.", k=8)                  # narrative
_ = chat_once("And what's the market?", k=8)                # one-liner via flexible prompt


/tmp/ipython-input-1698312562.py:28: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model=LLM_MODEL, temperature=0)
/tmp/ipython-input-1698312562.py:113: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


— Stateless QA —
🧠 Answer:
 Market: PACIFIC NORTHWEST 

📚 Sources:
[1] {'CivilSupplier': 'MasTec', 'ID': '12011', 'True Handover Status': 'Pending Handover > 5 Days', 'ClosedOut': 'Pending', 'SSV Hold ( Removal) Date': '', 'SwapType': 'Warm', 'Market': 'PACIFIC NORTHWEST', 'CutoverDate': '2025-07-29 00:00:00', '__sheet': 'Sheet1', 'Region': 'WEST', 'Usid': '59919', '__file': 'Combined_Violations.xlsx', 'Completed': 'True', '_doc_id': '59919', 'Quad': '1', '_chunk_id': 'whole', 'SubMarket': 'WASHINGTON'} -> ## Keys
Usid: 59919 | ID: 12011 | Region: WEST | Market: PACIFIC NORTHWEST | SubMarket: WASHINGTON | True Handover Status: Pending Handover > 5 Days | CutoverDate: 2025-07-29 00:00:00 | CivilSupplier:...
🧠 Answer:
 The project associated with USID 59919 is experiencing delays due to issues with RET connections, as noted in the comments. The Turf Vendor has indicated that troubleshooting is in progress, with a forecasted resolution date of August 8. No additional detailed reasons or c

In [ ]:
# ✅ Do NOT include "ids" in include=
res = chroma_vs._collection.get(
    where={"Usid": "50539"},
    include=["metadatas", "documents"]  # ids are returned automatically
)
print("In Chroma:", len(res.get("ids", [])))
if res.get("ids"):
    print("ID:", res["ids"][0])
    print("Metadata:", res["metadatas"][0])
    print("Doc preview:", res["documents"][0][:300])
else:
    print("USID 50539 is NOT in the vector store (likely filtered out in Step 2.10).")


In Chroma: 1
ID: 97237893-06b9-4868-88fe-bc28f4895052
Metadata: {'_chunk_id': 'whole', 'ID': '11914', '__sheet': 'Sheet1', 'CutoverDate': '2025-07-26 00:00:00', '_doc_id': '50539', 'Region': 'WEST', 'Usid': '50539', 'Completed': 'True', 'Market': 'SOUTHWEST STATES', 'True Handover Status': 'Handed Over', 'Quad': '1', 'ClosedOut': 'Pending', 'CivilSupplier': 'MasTec', '__file': 'Combined_Violations.xlsx', 'SubMarket': 'LAS VEGAS', 'SSV Hold ( Removal) Date': '2025-08-11 00:00:00', 'SwapType': 'Warm'}
Doc preview: ## Keys
Usid: 50539 | ID: 11914 | Region: WEST | Market: SOUTHWEST STATES | SubMarket: LAS VEGAS | True Handover Status: Handed Over | CutoverDate: 2025-07-26 00:00:00 | CivilSupplier: MasTec | SwapType: Warm | Completed: True | ClosedOut: Pending

## Turf Vendor Additional Description/Comments
08/0


In [ ]:
# =========================
# STEP 5 — Gradio ChatInterface (conversational, memory, small-talk, number normalization)
# =========================
# If needed: %pip -q install gradio

import re
import gradio as gr
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# 1) One global chain with memory (reuse llm, retriever, qa_prompt, CONDENSE_PROMPT from Step 4)
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="question",
    output_key="answer",
    return_messages=True,
)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    condense_question_prompt=CONDENSE_PROMPT,          # keep numeric IDs intact
    combine_docs_chain_kwargs={"prompt": qa_prompt},   # your flexible one-liner/summary prompt
)

# 2) Track last active row filter (USID/ID) across turns
LAST_FILTER = {}  # e.g., {"Usid": "111732"} or {"ID": "10517"}

def _iter_user_text(history):
    """Yield user texts from Gradio ChatInterface history (supports tuples or dict messages)."""
    for item in history or []:
        if isinstance(item, dict):
            if item.get("role") == "user":
                yield item.get("content", "")
        else:
            if isinstance(item, (list, tuple)) and len(item) >= 1:
                yield item[0] or ""

def _update_active_filter(msg: str, history):
    """Return the best filter for this turn; update LAST_FILTER when we find one."""
    if '_build_filter_from_question' in globals():
        f = _build_filter_from_question(msg)
        if f:
            LAST_FILTER.clear(); LAST_FILTER.update(f)
            return f
    try:
        for text in reversed(list(_iter_user_text(history))):
            if '_build_filter_from_question' in globals():
                f2 = _build_filter_from_question(text)
                if f2:
                    LAST_FILTER.clear(); LAST_FILTER.update(f2)
                    return f2
    except Exception:
        pass
    return LAST_FILTER or None

# 3) Small-talk detection & replies
_SMALL_GREET = re.compile(
    r"\b(hi|hello|hey|good\s*(morning|afternoon|evening)|how\s*are\s*you|how'?s\s*it\s*going|what'?s\s*up)\b",
    re.IGNORECASE,
)
_SMALL_THANKS = re.compile(r"\b(thanks|thank\s*you|thx|much\s*appreciated)\b", re.IGNORECASE)
_SMALL_ACK = re.compile(r"^\s*(ok(ay)?|cool|great|awesome|nice|got\s*it|understood|roger|y(ep|up)|sure)\s*[.!]?\s*$", re.IGNORECASE)
_SMALL_BYE = re.compile(r"\b(bye|good\s*night|good\s*bye|see\s*(ya|you))\b", re.IGNORECASE)

def _looks_like_query(s: str) -> bool:
    s = s.lower()
    if re.search(r"\d{3,}", s):
        return True
    for kw in ["usid", "id ", "market", "region", "submarket", "civil", "handover", "cutover",
               "swap", "rf", "ssv", "ericsson", "vendor", "comments", "reasons", "status"]:
        if kw in s:
            return True
    return False

def _smalltalk_kind(msg: str) -> str | None:
    s = (msg or "").strip()
    if not s or _looks_like_query(s):
        return None
    if _SMALL_THANKS.search(s): return "thanks"
    if _SMALL_BYE.search(s):    return "bye"
    if _SMALL_GREET.search(s):  return "greet"
    if _SMALL_ACK.search(s):    return "ack"
    return None

def _smalltalk_reply(kind: str) -> str:
    if kind == "thanks":
        return ("You're welcome! If you'd like, ask for a summary for a USID "
                "(e.g., 'Summarize 111732') or a quick field like 'What market is 111732 in?'")
    if kind == "bye":
        return "Take care! Ping me anytime if you need another site summary."
    if kind == "greet":
        return ("Hi! I can summarize a site (e.g., 'Summarize 111732') or answer quick lookups "
                "like 'Who is the civil supplier for 111732?'. I’ll remember context.")
    if kind == "ack":
        return "👍 Got it. Ask me about another USID or a specific field when you're ready."
    return "How can I help next?"

# 4) Bare-number normalization (e.g., "11386" → "Summarize USID 11386")
_NUM_ONLY = re.compile(r"^\s*(\d{3,})\s*$")
def _normalize_message(msg: str) -> str:
    m = _NUM_ONLY.match(msg or "")
    if m:
        return f"Summarize USID {m.group(1)}"
    return msg

FALLBACK_LINE = "I don't know based on the provided context."

# 5) Chat handler: small talk, normalization, reuse active filter, retry if needed
def chat(message, history):
    msg = (message or "").strip()
    if not msg:
        return "_Please enter a question_"

    # Small talk bypasses RAG
    kind = _smalltalk_kind(msg)
    if kind:
        return _smalltalk_reply(kind)

    # Normalize bare numbers to a clear intent
    msg = _normalize_message(msg)

    # Anchor retrieval to the current/last USID/ID
    flt = _update_active_filter(msg, history)

    prev_kwargs = dict(conversation_chain.retriever.search_kwargs)
    try:
        kwargs = {"k": 8}
        if flt:
            kwargs["filter"] = flt
        conversation_chain.retriever.search_kwargs.update(kwargs)

        out = conversation_chain.invoke({"question": msg})
        ans = out.get("answer", "")

        # If we got the strict fallback but have a target USID/ID, retry with a stronger cue
        if ans.strip() == FALLBACK_LINE and flt:
            target = flt.get("Usid") or flt.get("ID") or ""
            if target:
                retry_q = f"Provide a concise narrative summary for USID {target} based strictly on the context."
                out = conversation_chain.invoke({"question": retry_q})
                ans = out.get("answer", "") or ans

        return ans
    except Exception as e:
        return f"_Error: {e}_"
    finally:
        conversation_chain.retriever.search_kwargs = prev_kwargs

# 6) UI
examples = [
    "Tell me about USID 95149",
    "11386",
    "Who is the civil supplier?",
    "Any Ericsson RF details or civil comments?",
]

chatbot = gr.Chatbot(show_copy_button=True)

try:
    demo = gr.ChatInterface(
        fn=chat,
        chatbot=chatbot,
        title="AT&T Allagi Deployment — Conversational AI",
        description="Ask about USIDs, markets, suppliers, handover status. I remember context within this chat.",
        examples=examples,
        type="messages",
    )
except TypeError:
    demo = gr.ChatInterface(
        fn=chat,
        chatbot=chatbot,
        title="AT&T Allagi Deployment — Conversational AI",
        description="Ask about USIDs, markets, suppliers, handover status. I remember context within this chat.",
        examples=examples,
    )

demo.launch(share=True)


/tmp/ipython-input-3745682891.py:160: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(show_copy_button=True)
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:321: UserWarning: The type of the gr.Chatbot does not match the type of the gr.ChatInterface.The type of the gr.ChatInterface, 'messages', will be used.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://11f0d3db3be36655b3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
